In [33]:
import numpy as np
import cvxpy as cp
import pandas as pd
import matplotlib.pyplot as plt
import sys
sys.path.append('./utils')
from interval_bounds import *
from data_generation import *
from ridge_poisoning import *
from spectral_norm_constrained import solve_spectral_norm_constrained
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Create dataset

In [34]:
n = 100 # number of data points
m = 10 # number of features
sigma = 0.1
lamb = 10
rho = 1
X, y, w = gen_synthetic_normal(n,m,sigma)

In [35]:
#U_upper, U_lower = find_interval_bounds_infty_uncertainty(X, rho)

In [36]:
l = lambda X: np.linalg.norm(X@np.linalg.pinv(X)@y-y)**2

In [37]:
print("Shape of X: ", X.shape)
print("Shape of y: ", y.shape)
print("Shape of w: ", w.shape)

Shape of X:  (100, 10)
Shape of y:  (100, 1)
Shape of w:  (10, 1)


## Nominal Problem

In [38]:
loss, w_hat = solve_nominal(X,y,lamb=10)

In [39]:
print("Loss: ", loss)
print(l(X))

Loss:  129.47074495805606
1.018378166730138


## Robust Problem

In [40]:
loss, w_hat = solve_robust(X,y,rho=rho,lambda_=lamb)

In [41]:
print("Loss: ", loss)

Loss:  1201.5393063478427


## Poisoned Problem

In [42]:
print("Shape of X: ", X.shape)
print("Shape of y: ", y.shape)
print("Shape of w: ", w.shape)

Shape of X:  (100, 10)
Shape of y:  (100, 1)
Shape of w:  (10, 1)


In [43]:
print((y.T@X).shape)
print((np.ones((m+1,m+1))[m:m+1,0:m]).shape)

(1, 10)
(1, 10)


In [44]:
X_hats = []
losses = []

In [45]:
U, Sigma, V = np.linalg.svd(X)

In [46]:
Sigma

array([12.91692294, 12.6639056 , 10.95314388, 10.66013591, 10.18821058,
        9.64519679,  9.19801751,  9.06845783,  7.86252754,  7.42088063])

In [68]:
eps_range = np.linspace(6.8, 13.1, 7)

In [69]:
eps_range

array([ 6.8 ,  7.85,  8.9 ,  9.95, 11.  , 12.05, 13.1 ])

In [71]:
r_stars = []
losses = []
X_hats = []
for rho in eps_range:
    print(rho)
    #_, X_hat, _ = solve_poisoned_relaxed_interval_bounded(X,y,lamb=lamb,rho=rho, U_lower = U_lower, U_upper = U_upper, mu=10)
    X_hat, r_star = solve_spectral_norm_constrained(X, y, rho)
    X_hats.append(X_hat)
    r_stars.append(r_star)
    losses.append(l(X_hat))

6.8
7.85
8.9
9.95
11.0
12.05
13.1
Can make zero matrix


In [72]:
r_stars

[array([1.65775232e+27]),
 array([-3.73422514e+14]),
 array([-1.99104174e+23]),
 array([8.44958634e+22]),
 array([3.74926777e+20]),
 array([2.62739426e+19]),
 None]

In [73]:
losses

[145.4744567363716,
 145.4744567363716,
 145.47445673637168,
 145.47445673637168,
 145.4744567363716,
 145.47445673637168,
 1257.1365861864294]

In [ ]:
print(losses)

In [ ]:
print(X.shape)

In [ ]:
plt.figure()
for i in range(X.shape[0]):
    plt.scatter(X[i],y[i])

In [ ]:
for X_hat in X_hats:
    plt.figure()
    plt.xlim(-3,3)
    plt.ylim(-3,3)
    for i in range(X_hat.shape[0]):
        plt.scatter(X_hat[i],y[i])

In [ ]:
for X_hat in X_hats:
    print(np.linalg.norm(X_hat-X,2))